In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# K Means

In [1]:
!pip install spacy
!pip install emoji --upgrade
!python -m spacy download en_core_web_lg

import pandas as pd
import spacy
import emoji
import re

# Load the spaCy model
nlp = spacy.load('en_core_web_lg', disable=["parser", "ner"])

# Load the dataset
path = ('/content/drive/MyDrive/Copy of Us_trending_data_2022_2023.csv')
df = pd.read_csv(path)

# Set display options
pd.set_option('display.max_colwidth', None)  # No truncation for title
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 15)  # Set the maximum number of rows to display

# Drop duplicates and unnecessary columns
df.drop_duplicates(subset=['title', 'channelTitle'], inplace=True)
df.drop(['channelId'], axis=1, inplace=True)  # Drop the 'channelId' column


df.head(20)

2023-12-19 06:23:00.295224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 06:23:00.295282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 06:23:00.300323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 06:23:00.317665: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 06:23:02.137287: W tensorflow/compiler/tf2

,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description
0,The Lawsuit of Shane Dawson,2022-02-15,shane,23,2022-02-16,shane|dawson|conspiracy|theories|journalism|investigative journalism|docuseries|documentary,1071503,77929,0,12215,"MERCH! New Mystery Pig Bags Available Now! https://www.shanedawsonmerch.com/ PODCAST IG @ShaneDawsonPodcast https://www.instagram.com/shanedawsonpodcast/?hl=enMy 2nd Youtube Channel https://www.youtube.com/channel/UCAVojJ1k03GZzjSbdXXunkwMy Instagram @Shanedawson https://www.instagram.com/shanedawson/?hl=enIn this video we have fun around the house, get deep about life’s toughest moments, and check to see if the pizza is still wonky! Enjoy!! Ryland’s Vlogs https://www.youtube.com/channel/UCVe_RrR9AMeY6Z8ixOv5gtwRyland’s Main Channel https://www.youtube.com/user/ryanadams7Ryland’s Podcast https://www.youtube.com/channel/UCH2c0ynrljAA-jW8tq3u3HARyland’s Instagram https://www.instagram.com/rylandadams/?hl=enChris’s Channel https://www.youtube.com/c/ChrisBstationChris’s Instagram https://www.instagram.com/chrisbstation/?hl=enMaxine Friend, The Psychic I Mentioned. She’s amazing!https://maxinefriend.com/https://www.instagram.com/maxinefriend/?hl=enMy Brother Jerid’s Instagram https://www.instagram.com/letsmindtravel/?hl=enJerid’s Channel https://www.youtube.com/c/LetsMindTravel/videosBusiness Contact:United Talent AgencyT: 310.273.6700"
1,When the quiet kid plays his music out loud 😂,2022-02-13,Onevilage,23,2022-02-16,Tiktok|Comedy|Foryou|Onevilage|Shorts|Hilarious|Jokes|Viral|Plot twist|Lol|Funny|Don’t flop|Recommend,2381552,289021,0,3069,🆂🆄🅱🆂🅲🆁🅸🅱🅴▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬FOLLOW THESE TO KEEP UP TO DATE:INSTAGRAM ► (onevilage) https://www.instagram.com/onevilage/DISCORD ► https://discord.io/onevilageSNAPCHAT ► (sahdude221): https://www.snapchat.com/add/sahdude221TWITTER ► (onevilage): https://twitter.com/onevilageTWITCH ► (onevilage): https://www.twitch.tv/onevilage▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬#onevilageDance credit: ashleykeno17 😂❤️e
2,Lizard Cupid 2 #shorts,2022-02-13,TheOdd1sOut,23,2022-02-16,[None],1050293,95261,0,1766,#shortsMatt Watts art ➤ https://www.instagram.com/mattwattsart/Emilee Dummer ➤ https://www.instagram.com/edummerart/ Kelly Jensen ➤ https://www.instagram.com/kelly_anne_art/Claire Anne ➤ https://www.instagram.com/claireanneillustrationAmelia Rosenbalm ➤ https://www.instagram.com/galloame/
3,how youtube is when you start watching a show,2022-02-14,Gus Johnson,23,2022-02-16,gus|gus johnson|gus meme|how youtube is when you start a show|meme|new meme|meme playlist|sketch comedy|short funny video|reddit|reddit trending|unusual memes|meme playlist funny|sven johnson|gus and sven|ian kung,292224,26736,0,2923,"please i just wanna watch the show↓COME FOLLOW ME HERE OR I WILL CRY (HARD)↓NEW MERCH: shop.gusjohnson.coGET LIVE TICKETS HERE: https://www.gusjohnson.co/tour- Twitch: https://www.twitch.tv/gusjohnson- Twitter: https://twitter.com/Gusbuckets- Instagram: https://www.instagram.com/johnsongus- Facebook: https://www.facebook.com/gustoonz- Reddit: https://www.reddit.com/r/gusjohnsonGus Johnson plays all sorts of characters, from Mitchell Robbins to JK Rowling to some guy in God's Country to Imbiamba Jombes to filming videos with his mom to pillow guy to the my pool guy to small town reviews to how they use shotguns in movies to the tough kid growing up to your friend who has a knife to spotify ad guy to the how they use shotgun to the gus johnson shotgun to if i was frozone... I don't know why I am writing this in third person. I am Gus. I wrote this. My brother is Sven Johnson, that guy's alright. Guys, if you put all this dumb stuff in the description, it helps your videos. isn't that wild?Thanks for watching and sharing! Don't stab people. I'll see you later."
4,The Metaverse is So Stupid,2022-02-15,Eddy Burback,23,2022-02-16,[None],434369,43331,0,3398,my other channel - https://www.youtube.com/channel/UCJHM5IJ44__fkOxkD4vxoBwmy tw

In [2]:
import random

#print(df.head())
# Define regex pattern for a broader range of emojis
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"   # Enclosed characters
                           "]+", flags=re.UNICODE)
custom_emoji_map = {
    # Add your specific emojis and their text representations here
    "🤓": "nerd_face",
    "☝️": "index_pointing_up",
    "😂": "Face_with_Tears_of_Joy",
    "😭":"Loudly_Crying_Face",
    "💔":"Broken_Heart",
    "😩":"Tired_Face",
    "🔵": "blue_cirlce",
    "🟠":"orange_circle",
    "💨":"zoom_smoke",
    "😱":"Face_Screaming_in_Fear",
    "🙄":"Face_with_Rolling_Eyes",
    "💅":"nails",
    "😍":"smiling_face_with_heart_eyes",
    "✈️":"airplane"
}

# Filter for Shorts
shorts_keywords = ['#Shorts', '#short', 'Shorts', 'shorts']
shorts_pattern = '|'.join(shorts_keywords)
df = df[~df['title'].str.contains(shorts_pattern, na=False) &
        ~df['description'].str.contains(shorts_pattern, na=False) &
        ~df['channelTitle'].str.contains(shorts_pattern, na=False)]


# Batch processing function with custom emoji encoding
def process_text_batch(texts, batch_size=500):
    processed_texts = []
    for text in nlp.pipe(texts, batch_size=batch_size):
        # Ensure text is a string
        if not isinstance(text, str):
            text = str(text)

        # Remove '|' character
        text = text.replace('|', ' ')

        # Custom emoji replacement
        for emoji_char, emoji_text in custom_emoji_map.items():
            text = text.replace(emoji_char, f" {emoji_text} ")

        # Replace emojis using regex pattern
        text_with_emojis_encoded = emoji_pattern.sub(lambda x: f" {x.group(0)} ", text)
        processed_text = ' '.join([token.lemma_ for token in nlp(text_with_emojis_encoded) if not token.is_stop and not token.is_punct])
        processed_texts.append(processed_text)
    return processed_texts


def clean_tags(tags_string):
    if pd.isna(tags_string):
        return ""
    return ' '.join([tag.strip() for tag in tags_string.split('|')])

# Apply batch processing for titles and descriptions
df['processed_title'] = process_text_batch(df['title'].astype(str).tolist())
#df['processed_description'] = process_text_batch(df['description'].astype(str).tolist())

# Apply tag cleaning
df['cleaned_tags'] = df['tags'].apply(clean_tags)

# Applying tag cleaning and trimming spaces in 'cleaned_tags', 'title', and 'processed_title'
df['cleaned_tags'] = df['tags'].apply(clean_tags).str.strip()
df['title'] = df['title'].str.strip()
df['processed_title'] = df['processed_title'].str.strip()

# Display the first 15 rows of the DataFrame
print("First 20 Rows:")
print(df.iloc[:100, [df.columns.get_loc(c) for c in ('title', 'processed_title', 'cleaned_tags')]])


print("______________________________________________________________________________________________________________\n")
# Print the number of rows in the DataFrame
print("Total number of rows in DataFrame:", df.shape[0])

# Choose three random start points to print 15-record samples
random_starts = random.sample(range(len(df) - 25), 3)

# Print the 15-record samples from each start point
for start in random_starts:
    print(f"Sample Records starting from index {start}:")
    print(df.iloc[start:start+15, [df.columns.get_loc(c) for c in ('title', 'processed_title', 'cleaned_tags', 'tags')]])
    print("\n---\n")


# Print the number of rows in the DataFrame
print("Total number of rows in DataFrame:", df.shape[0])

# Print the 15-record samples from each start point
#for start in random_starts:
    #print(f"Sample Records starting from index {start}:")
    #print(df.iloc[start:start+15])
    #print("\n---\n")
# Optionally, save to a file
#df.to_csv('/content/drive/My Drive/processed_data.csv', index=False)

First 20 Rows:
                                                              title                                             processed_title                                                                                                                                                                                                           cleaned_tags
0                                       The Lawsuit of Shane Dawson                                        Lawsuit Shane Dawson                                                                                                                            shane dawson conspiracy theories journalism investigative journalism docuseries documentary
1                     When the quiet kid plays his music out loud 😂          quiet kid play music loud   face_with_tears_of_joy                                                                                                                  Tiktok Comedy Foryou Onevilage Shorts Hilarious Jokes Vira

In [3]:
# Function to identify unencoded emojis
def find_unencoded_emojis(original, processed):
    # Find all emojis in the original text
    original_emojis = emoji_pattern.findall(original)
    # Convert each emoji to its text representation
    encoded_emojis = []
    for e in original_emojis:
        if e in custom_emoji_map:
            # If emoji is in custom map, use that encoding
            encoded_emojis.append(custom_emoji_map[e])
        else:
            # Otherwise, use standard emoji encoding
            encoded_emojis.append(emoji.demojize(e, delimiters=(""," ")))
    # Identify emojis not encoded in the processed text
    unencoded = [e for e in original_emojis if not any(encoded in processed for encoded in encoded_emojis)]
    return unencoded

# Apply the function to the DataFrame
df['unencoded_emojis'] = df.apply(lambda x: find_unencoded_emojis(x['title'], x['processed_title']), axis=1)







#extract records with no emojis in title
df = df[df['unencoded_emojis'].apply(lambda x: len(x) == 0)]


Titles with Unencoded Emojis:
Total number of rows in DataFrame: 24515


In [4]:
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [5]:
#convert to lowercase
df['processed_title'] = df['processed_title'].str.lower()
df['cleaned_tags'] = df['cleaned_tags'].str.lower()
print(df['cleaned_tags'])
print(df['processed_title'])

0                                                                                                                                                                                                                                                                                                                                                                                  shane dawson conspiracy theories journalism investigative journalism docuseries documentary
3                                                                                                                                                                                                                                                        gus gus johnson gus meme how youtube is when you start a show meme new meme meme playlist sketch comedy short funny video reddit reddit trending unusual memes meme playlist funny sven johnson gus and sven ian kung
4                                                                         

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

#use vectorizer only considering words used twice
CV = CountVectorizer(min_df=2)
X = CV.fit_transform(df['processed_title']).toarray()

# X_cleaned_tags = CV.fit_transform(df['cleaned_tags']).toarray()

# X_category = df['categoryId'].astype(int).to_numpy().reshape(-1, 1)
# X_likes = df['likes'].to_numpy().reshape(-1, 1)

# X = np.concatenate((X_processed_title, X_category), axis=1)

y = df['categoryId'].to_numpy().reshape(-1, 1)
print(np.shape(X))
print(np.shape(y))

(24515, 10146)
(24515, 1)


In [11]:


from sklearn.model_selection import train_test_split

#split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.naive_bayes import GaussianNB
# there are different models we could use here
model = GaussianNB()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, precision_score

#accuracy, f1_score, and precision metrics
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='macro'))
print(precision_score(y_test, y_pred, average='macro'))





/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5594533958800735
0.3986687386030324
0.38497484347908384


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
